<a href="https://colab.research.google.com/github/hyelog/2022_Samsung_DL/blob/main/%5BAI_Essential%5D_Streamlit%EA%B3%BC_langchain%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%B1%97%EB%B4%87_%EA%B5%AC%ED%98%84_%EC%8B%A4%EC%8A%B5_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [AI Essential] 4일차 LCEL Workshop
## Streamlit과 langchain을 이용한 챗봇 구현

### 필수 패키지 설치

In [ ]:
%%capture
!pip install langchain langchain_community langchain_openai streamlit pyngrok -qU

### NGROK API Key 설정

In [ ]:
PYNGROK_API_KEY = '2rNnTWy7dlvRWFCzizHoKwY222E_7pNcHaAZravgzZa9Q7BrV'

### Base Page 만들기
 - 제목이 "가전제품 고객지원 봇"인 스트림릿 페이지를 생성하세요.

In [ ]:
%%writefile app.py
import streamlit as st

# 제목 입력
st.title("가전제품 고객지원 봇")

Overwriting app.py


 - 스트림릿 서버 실행 후 생성 결과를 확인하세요.

In [ ]:
# 그대로 실행하세요.
from pyngrok import ngrok

# ngrok 서비스 인증
ngrok.set_auth_token(PYNGROK_API_KEY)

# app1.py 백그라운드 프로세스 실행
!nohup streamlit run app.py --server.port 5011 &

# ngrok 터널링 실행
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# ngrok 터널링 결과
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://9d4a-34-73-229-250.ngrok-free.app


### 사이드 바 추가하기
 - 사이드 바를 추가하세요.
 - 사이드 바에 OPENAI API Key를 입력 받기 위한 텍스트 입력기를 추가하세요.
   - Key 입력 시 노출되지 않도록 비밀번호 입력 형식으로 생성하세요.
 - 페이지 재생성 후 이전 웹페이지에서 F5를 눌러 반영된 결과를 확인하세요.

In [ ]:
%%writefile app.py
import streamlit as st

# 사이드 바 추가 및 텍스트 입력기 추가
st.sidebar.text_input('OPENAI API KEY', type='password')

# 제목 출력
st.title("가전제품 고객지원 봇")

Overwriting app.py


 - 이제 제목 "가전제품 고객지원 봇"은 기본적으로 출력하지 않습니다.
 - 텍스트 입력기에 값이 입력되면 메인 페이지에 "가전제품 고객지원 봇"이 제목으로 출력되도록 페이지를 수정하세요.
 - 페이지 재생성 후 이전 웹페이지에서 F5를 눌러 반영된 결과를 확인하세요.


In [ ]:
%%writefile app.py
import streamlit as st

# 사이드 바 추가 및 텍스트 입력기 추가
key = st.sidebar.text_input('OPENAI API KEY', type='password')

# key 값이 있는 경우만
if key:
    # 제목 출력
    st.title("가전제품 고객지원 봇")

Overwriting app.py


### Key 확인 및 LLM Chain 구성하기
 - 입력된 Key가 유효한 Key인지 확인하는 코드를 작성하세요.
 - 유효한 경우 제목 "가전제품 고객지원 봇"을 출력합니다.
 - 페이지 재생성 후 이전 웹페이지에서 F5를 눌러 반영된 결과를 확인하세요.

In [ ]:
%%writefile app.py
import streamlit as st
from langchain_openai import ChatOpenAI

# OpenAI API Key 설정
OPENAI_API_KEY = ''

# api key 유효성 체크 (그대로 실행하세요)
def validate_openai_api_key(key):
    try:
        llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, openai_api_key=key)
        llm.invoke('key테스트입니다.')
        return True
    except Exception as e:
        if e.code == 'invalid_api_key':
            st.sidebar.error('OPENAI API 키를 확인해주세요.')
        else:
            st.sidebar.error(e.code)
        return False

# 사이드 바 추가 및 텍스트 입력기 추가
key = st.sidebar.text_input('OPENAI API KEY', type='password', value=OPENAI_API_KEY)

# key 값이 있는 경우만
if key:
    # key가 유효한 경우만 제목 출력
    if validate_openai_api_key(key):
        st.title("가전제품 고객지원 봇")

Overwriting app.py


 - LLM Chain을 구성하여 페이지에 추가하세요.
 - Chain은 Prompt, LLM, Parser로 구성하세요.
 - Prompt는 제품 고장에 대해 문의할 수 있도록 간단하게 구성하세요.
 - st.chat_input을 이용하여 사용자 문의사항을 입력 받으세요.
  ```
        # 예시
        message = st.chat_input('메시지를 입력하세요.')
  ```
 - 입력 값이 있는 경우 LLM을 이용하여 답변을 생성하고 사용자 입력과 LLM 응답을 st.chat_message를 이용하여 출력하세요.
   ```
        # 예시
        with st.chat_message('user'):
            st.write(message)
        with st.chat_message('assistant'):
            st.write(message)
   ```
 - 페이지 재생성 후 이전 웹페이지에서 F5를 눌러 반영된 결과를 확인하세요.

In [ ]:
%%writefile app.py
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# OpenAI API Key 설정
OPENAI_API_KEY = ''

# api key 유효성 체크 (그대로 실행하세요)
def validate_openai_api_key(key):
    try:
        llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, openai_api_key=key)
        llm.invoke('key테스트입니다.')
        del llm
        return True
    except Exception as e:
        if e.code == 'invalid_api_key':
            st.sidebar.error('OPENAI API 키를 확인해주세요.')
        else:
            st.sidebar.error(e.code)
        del llm
        return False

# 사이드 바 추가 및 텍스트 입력기 추가
key = st.sidebar.text_input('OPENAI API KEY', type='password', value=OPENAI_API_KEY)

# key 값이 있는 경우만 출력
if key:
    # key가 유효한 경우만 제목 출력
    if validate_openai_api_key(key):
        template = """당신은 가전제품 고객지원을 담당하는 챗봇입니다. 고객의 문의에 친절하고 상세하게 답해주세요. 가전제품과 관련 없는 질문은 답하지 마세요.
user: {user_input}"""
        # 프롬프트 생성
        prompt = PromptTemplate.from_template(template)
        # LLM 생성
        llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, openai_api_key=key)
        # 파서 생성
        parser = StrOutputParser()
        # 체인 생성
        chain = (prompt | llm | parser)

        # 제목 출력
        st.title("가전제품 고객지원 봇")

        if "message" not in st.session_state:
            st.session_state.messages = []
        for message in st.session_state.messages:
            with st.chat_messae(message["role"]):
                st.write(message["content"])

        # 챗 유저 메시지 입력
        if question := st.chat_input("고장 사항을 입력하세요:"):
            # 챗 유저 메시지 출력
            with st.chat_message("user"):
                st.write(question)
            # 챗 도우미 메시지 출력
            with st.chat_message("assistant"):
                # LLM 체인을 이용하여 유저 메시지에 대한 도우미 메시지 생성 후 출력
                history = '\n'.join([':'.join(entry.values()) for entry in st.session_state.messages])

                message = st.write_stream(chain.stream({"user_input": question, "history": history}))
                st.write(message)
            st.session_state.messages.append({"role": "user", "content": question})
            st.session_state.messages.append({"role": "assistant", "content": message})


# 자동 스크롤 스크립트 (그대로 사용하세요.)
js = '''
<script>
    var body = window.parent.document.querySelector(".main");
    console.log(body);
    body.scrollTop = 0;
</script>
'''

st.components.v1.html(js)

Overwriting app.py


### 서비스 종료 및 터널링 종료

In [ ]:
# Streamlit 서비스 종료
!pkill -f streamlit
# ngrok 터널링 종료
ngrok.disconnect(ngrok_tunnel.public_url)